In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ortho_group

np.random.seed(0)
x = ortho_group.rvs(6)

In [4]:
std_list = [1.5, 2.0, 1.5, 2.5, 1.7, 2.2]
fdf = pd.DataFrame()
for i in range(6):
    fdf[i] = np.random.normal(loc=0.0, scale=std_list[i], size=200000)

In [5]:
print(x[0])
print(x[1])

[ 0.47669462  0.15964814 -0.47766656  0.55284255 -0.13930486  0.44053222]
[ 0.10813329 -0.8838343   0.06718921  0.39309219  0.13141423 -0.1756091 ]


In [6]:
fdf["group"] = 0.9 / (1.0 + np.exp(-np.matmul(fdf.values, x[0].T) + 5.0)) +\
               0.1 / (1.0 + np.exp(-np.matmul(fdf.values, x[1].T) + 6.0))

In [7]:
fdf["prob"] = np.random.uniform(low=0.0, high=1.0, size=200000)
fdf["label"] = fdf["group"] > fdf["prob"]
fdf["label"] = fdf["label"] * 1

In [8]:
fdf.label.value_counts()

0    193851
1      6149
Name: label, dtype: int64

In [9]:
cdf = fdf.copy()
Y = cdf.label.values
X = cdf.drop(["prob", "label", "group"], axis=1).values
print(X)

[[-3.2644018   2.16129832  1.32745473 -0.92475887 -0.84604962  0.38897996]
 [ 1.80863845  0.9712237  -1.26000599  0.38868967  0.4564715   1.39185836]
 [ 0.06042143 -1.84251117  0.79550194 -4.35093213  1.54590996  2.81069942]
 ...
 [ 2.44711711  1.14586658  2.67928386 -0.28663962  0.36477659 -0.5870959 ]
 [ 1.13644146  2.48149056 -0.24881862  0.07782861 -1.73102276 -1.25522277]
 [ 1.18222992  2.00613358  2.25983241 -1.2610145  -0.83184001 -0.58121851]]


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
torch.manual_seed(0)

In [40]:
class ImbalancedDatasetSampler(Data.sampler.Sampler):
    def __init__(self, dataset, indices=None, num_samples=None):
        self.indices = list(range(len(dataset))) if indices is None else indices
        self.num_samples = len(self.indices) if num_samples is None else num_samples
        
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1
        
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)] for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)
    
    def _get_label(self, dataset, idx):
        return dataset[idx][1].item()
    
    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(self.weights, self.num_samples, replacement=True))
    
    def __len__(self):
        return self.num_samples

In [14]:
class SubNet(nn.Module):
    def __init__(self):
        super(SubNet, self).__init__()
        self.fc1 = nn.Linear(1, 10)
        self.fc2 = nn.Linear(10, 2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [15]:
class FrontProcessing(nn.Module):
    def __init__(self):
        super(FrontProcessing, self).__init__()
        self.in1 = nn.Linear(6, 2)
        
    def forward(self, x):
        x = self.in1(x)
        return x
    

In [22]:
class Net(nn.Module):
    def __init__(self, FrontProcessing, SubNet):
        super(Net, self).__init__()
        self.front = FrontProcessing()
        self.subnet1 = SubNet()
        self.subnet2 = SubNet()
    
    def forward(self, x):
        x = self.front(x)
        x1 = x[:, 0].view(-1, 1)
        x2 = x[:, 1].view(-1, 1)
        
        x1 = self.subnet1(x1)
        x2 = self.subnet2(x2)
        x = (x1 + x2) / 2.0
        return x

In [27]:
net = Net(FrontProcessing, SubNet)
net.double()
print(net)

Net(
  (front): FrontProcessing(
    (in1): Linear(in_features=6, out_features=2, bias=True)
  )
  (subnet1): SubNet(
    (fc1): Linear(in_features=1, out_features=10, bias=True)
    (fc2): Linear(in_features=10, out_features=2, bias=True)
  )
  (subnet2): SubNet(
    (fc1): Linear(in_features=1, out_features=10, bias=True)
    (fc2): Linear(in_features=10, out_features=2, bias=True)
  )
)


In [28]:
X_torch = torch.DoubleTensor(X)
Y_torch = torch.DoubleTensor(Y)

In [29]:
net(X_torch.double())

tensor([[ 0.0684, -0.2165],
        [ 0.0400, -0.2849],
        [ 0.2150, -0.2697],
        ...,
        [-0.2293, -0.1807],
        [-0.0510, -0.1756],
        [-0.1764, -0.1782]], dtype=torch.float64, grad_fn=<DivBackward0>)

In [41]:
torch_data_set = Data.TensorDataset(X_torch, Y_torch)
sampler = ImbalancedDatasetSampler(torch_data_set)
loader = Data.DataLoader(dataset=torch_data_set, batch_size=10000, sampler=sampler)

In [37]:
list(net.parameters())

[Parameter containing:
 tensor([[ 0.3867,  0.2593,  0.3876, -0.0295, -0.3667, -0.1935],
         [ 0.2780, -0.0026, -0.2029, -0.3128, -0.3821, -0.3446]],
        dtype=torch.float64, requires_grad=True), Parameter containing:
 tensor([-0.0828,  0.2239], dtype=torch.float64, requires_grad=True), Parameter containing:
 tensor([[ 0.5406],
         [-0.9644],
         [ 0.6238],
         [-0.7825],
         [-0.2114],
         [-0.4055],
         [-0.1926],
         [-0.1963],
         [-0.8973],
         [-0.8634]], dtype=torch.float64, requires_grad=True), Parameter containing:
 tensor([-0.1565,  0.0129, -0.4543,  0.3767, -0.9001, -0.0675,  0.8794, -0.4079,
          0.9030,  0.3622], dtype=torch.float64, requires_grad=True), Parameter containing:
 tensor([[-0.2854,  0.2001, -0.0365, -0.1412,  0.2529, -0.2555,  0.0339, -0.0662,
           0.2258,  0.0883],
         [ 0.1519,  0.1117, -0.0760, -0.0665, -0.2606,  0.1713,  0.2511,  0.2164,
          -0.2231,  0.0141]], dtype=torch.float64, 

In [42]:
lr = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=lr)

In [43]:
for epoch in range(600):
    for step, (X_train, Y_train) in enumerate(loader):
        optimizer.zero_grad()
        X_train = X_train.requires_grad_(True)
        
        prediction = net(X_train)
        loss = criterion(prediction, Y_train.long())
        
        loss.backward()
        optimizer.step()
        
    with torch.no_grad():
        loss_all = criterion(net(X_torch), Y_torch.long())
        print("Epoch: ", epoch, "| loss: ", loss_all.data.numpy())
        print("##########################")

Epoch:  0 | loss:  0.5827303760328246
##########################
Epoch:  1 | loss:  0.5989591187351524
##########################
Epoch:  2 | loss:  0.6110256671136713
##########################
Epoch:  3 | loss:  0.6186280468063902
##########################
Epoch:  4 | loss:  0.6224400316661297
##########################
Epoch:  5 | loss:  0.6243678645755922
##########################
Epoch:  6 | loss:  0.6242167627464832
##########################
Epoch:  7 | loss:  0.6231564534524688
##########################
Epoch:  8 | loss:  0.6196971120763054
##########################
Epoch:  9 | loss:  0.6157556431852632
##########################
Epoch:  10 | loss:  0.6113481476553692
##########################
Epoch:  11 | loss:  0.6064447880078464
##########################
Epoch:  12 | loss:  0.6012611085124643
##########################
Epoch:  13 | loss:  0.5950852829790793
##########################
Epoch:  14 | loss:  0.5893902881640618
##########################
Epoch:  15 | loss:  

KeyboardInterrupt: 

In [44]:
for param in net.parameters():
    print(param)


Parameter containing:
tensor([[ 0.6029,  0.1175, -0.0483,  0.1309, -0.4506, -0.0784],
        [-0.1818,  0.0059,  0.5347, -0.6485, -0.1501, -0.5938]],
       dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([-0.2021,  0.3019], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[ 0.6069],
        [-0.9742],
        [ 0.6284],
        [-0.7815],
        [-0.2085],
        [-0.3799],
        [-0.1687],
        [-0.1953],
        [-0.9068],
        [-0.8715]], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([-0.1560,  0.0284, -0.4538,  0.3816, -0.9007, -0.0953,  0.8710, -0.4086,
         1.0051,  0.3842], dtype=torch.float64, requires_grad=True)
Parameter containing:
tensor([[-0.3596,  0.2643, -0.1116, -0.0536,  0.2528, -0.2364,  0.1515, -0.0665,
          0.3968,  0.1800],
        [ 0.2262,  0.0474, -0.0010, -0.1540, -0.2605,  0.1522,  0.1335,  0.2166,
         -0.3940, -0.0776]], dtype=torch.float64, requires_grad=True)
Param

In [49]:
out = net(X_torch)
_, prediction = torch.max(out.data, 1)
print('accuracy:%.3f %%' % (100 * torch.sum(Y_torch.long()==prediction) / 200000))


accuracy:79.000 %
